In [3]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
NSE Index Constituents Fetcher (Simplified + Symbol Array)
==========================================================

Fetches the latest constituent list for any NSE index (NIFTY 50, 500, etc.)
and saves it into a CSV file with columns: symbol,name

Also prints the list of symbols (with ".NS" suffix) as a Python array.

Example output:
---------------
✅ Saved 500 records to: nifty500_symbols.csv

Sample records:
     symbol                         name
  RELIANCE.NS         Reliance Industries
   HDFCBANK.NS                    HDFC Bank
        TCS.NS  Tata Consultancy Services

Python Array:
['RELIANCE.NS', 'HDFCBANK.NS', 'TCS.NS', ...]
"""

import requests
import pandas as pd
from io import StringIO
import os

# ==============================================================
# 1. CONFIGURATION
# ==============================================================

# Choose one of the following URLs:
# CSV_URL = "https://nsearchives.nseindia.com/content/indices/ind_nifty50list.csv"
# CSV_URL = "https://nsearchives.nseindia.com/content/indices/ind_niftynext50list.csv"
# CSV_URL = "https://nsearchives.nseindia.com/content/indices/ind_niftybanklist.csv"
CSV_URL = "https://nsearchives.nseindia.com/content/indices/ind_nifty500list.csv"
# CSV_URL = "https://nsearchives.nseindia.com/content/indices/ind_niftymidcap100list.csv"
# CSV_URL = "https://nsearchives.nseindia.com/content/indices/ind_niftysmallcap100list.csv"

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.nseindia.com/market-data/live-market-indices"
}

# ==============================================================
# 2. FETCH AND PARSE
# ==============================================================

session = requests.Session()
session.headers.update(HEADERS)

# Warm-up to get cookies
session.get("https://www.nseindia.com", timeout=5)

print(f"Fetching data from: {CSV_URL}")
response = session.get(CSV_URL, timeout=10)
response.raise_for_status()

df = pd.read_csv(StringIO(response.text))

# ==============================================================
# 3. CLEAN AND FORMAT
# ==============================================================

df.columns = [c.strip().title() for c in df.columns]
if "Symbol" not in df.columns or "Company Name" not in df.columns:
    raise ValueError("Expected columns not found in NSE CSV response.")

df["symbol"] = df["Symbol"].astype(str) + ".NS"
df["name"] = df["Company Name"]

# ==============================================================
# 4. SAVE OUTPUT
# ==============================================================

basename = os.path.basename(CSV_URL).replace("ind_", "").replace("list.csv", "")
outfile = f"{basename}_symbols.csv"

df_out = df[["symbol", "name"]]
df_out.to_csv(outfile, index=False)
print(f"\n✅ Saved {len(df_out)} records to: {outfile}")

# ==============================================================
# 5. PRINT OUTPUT
# ==============================================================

# Preview first few rows
print("\nSample records:")
print(df_out.head(10).to_string(index=False))

# Print Python array of symbols
symbols = df_out["symbol"].tolist()
print("\nPython Array:")
print(symbols)


Fetching data from: https://nsearchives.nseindia.com/content/indices/ind_nifty500list.csv

✅ Saved 503 records to: nifty500_symbols.csv

Sample records:
      symbol                        name
   360ONE.NS            360 ONE WAM Ltd.
  3MINDIA.NS               3M India Ltd.
      ABB.NS              ABB India Ltd.
      ACC.NS                    ACC Ltd.
ACMESOLAR.NS    ACME Solar Holdings Ltd.
   AIAENG.NS        AIA Engineering Ltd.
APLAPOLLO.NS       APL Apollo Tubes Ltd.
   AUBANK.NS  AU Small Finance Bank Ltd.
      AWL.NS      AWL Agri Business Ltd.
AADHARHFC.NS Aadhar Housing Finance Ltd.

Python Array:
['360ONE.NS', '3MINDIA.NS', 'ABB.NS', 'ACC.NS', 'ACMESOLAR.NS', 'AIAENG.NS', 'APLAPOLLO.NS', 'AUBANK.NS', 'AWL.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABBOTINDIA.NS', 'ACE.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'ATGL.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AF

In [2]:
import requests
import pandas as pd
from io import StringIO

INDEX_URLS = [
    "https://nsearchives.nseindia.com/content/indices/ind_nifty500list.csv",
    # "https://nsearchives.nseindia.com/content/indices/ind_niftymidcap100list.csv",
    # "https://nsearchives.nseindia.com/content/indices/ind_niftysmallcap100list.csv",
]

HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "en-US,en;q=0.9",
    "Referer": "https://www.nseindia.com/market-data/live-market-indices"
}

def fetch_symbols(session: requests.Session, url: str) -> list[str]:
    """Fetch a CSV from NSE and return a list of uppercased symbols."""
    resp = session.get(url, timeout=15)
    resp.raise_for_status()
    df = pd.read_csv(StringIO(resp.text))
    symbol_col = None
    for col in df.columns:
        if col.strip().lower() in {"symbol", "symbols"}:
            symbol_col = col
            break
    if symbol_col is None:
        raise ValueError(f"Could not find 'Symbol' column in CSV: {url}")
    syms = (
        df[symbol_col]
        .astype(str)
        .str.strip()
        .str.upper()
        .tolist()
    )
    return syms
session = requests.Session()
session.headers.update(HEADERS)
session.get("https://www.nseindia.com", timeout=10)
all_syms = []
for url in INDEX_URLS:
    try:
        all_syms.extend(fetch_symbols(session, url))
    except Exception as e:
        print(f"[WARN] Failed to fetch {url}: {e}")
unique_syms = list(dict.fromkeys(all_syms))
final_list = [s if s.endswith(".NS") else f"{s}.NS" for s in unique_syms]
final_list = sorted(final_list)
print(final_list)


['360ONE.NS', '3MINDIA.NS', 'AADHARHFC.NS', 'AARTIIND.NS', 'AAVAS.NS', 'ABB.NS', 'ABBOTINDIA.NS', 'ABCAPITAL.NS', 'ABFRL.NS', 'ABLBL.NS', 'ABREL.NS', 'ABSLAMC.NS', 'ACC.NS', 'ACE.NS', 'ACMESOLAR.NS', 'ADANIENSOL.NS', 'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS', 'AEGISLOG.NS', 'AEGISVOPAK.NS', 'AFCONS.NS', 'AFFLE.NS', 'AGARWALEYE.NS', 'AIAENG.NS', 'AIIL.NS', 'AJANTPHARM.NS', 'AKUMS.NS', 'AKZOINDIA.NS', 'ALKEM.NS', 'ALKYLAMINE.NS', 'ALOKINDS.NS', 'AMBER.NS', 'AMBUJACEM.NS', 'ANANDRATHI.NS', 'ANANTRAJ.NS', 'ANGELONE.NS', 'APARINDS.NS', 'APLAPOLLO.NS', 'APLLTD.NS', 'APOLLOHOSP.NS', 'APOLLOTYRE.NS', 'APTUS.NS', 'ARE&M.NS', 'ASAHIINDIA.NS', 'ASHOKLEY.NS', 'ASIANPAINT.NS', 'ASTERDM.NS', 'ASTRAL.NS', 'ASTRAZEN.NS', 'ATGL.NS', 'ATHERENERG.NS', 'ATUL.NS', 'AUBANK.NS', 'AUROPHARMA.NS', 'AWL.NS', 'AXISBANK.NS', 'BAJAJ-AUTO.NS', 'BAJAJFINSV.NS', 'BAJAJHFL.NS', 'BAJAJHLDNG.NS', 'BAJFINANCE.NS', 'BALKRISIND.NS', 'BALRAMCHIN.NS', 'BANDHANBNK.NS', 'BANKBARODA.NS', 'BANKINDIA.NS', '

# NIFTY 200 Momentum Stocks 30

In [12]:
import pandas as pd
import requests
from io import StringIO
import ssl

URL = "https://www.niftyindices.com/IndexConstituent/ind_nifty200Momentum30_list.csv"

# Use the requests session that's already set up in previous cells
response = session.get(URL, verify=False, timeout=15)
response.raise_for_status()

# Read from the response content
df = pd.read_csv(StringIO(response.text))

# Find the symbol column (varies; usually 'Symbol' or first col)
sym_col = next((c for c in df.columns if str(c).strip().lower().startswith("symbol")), df.columns[0])

# Make the .NS list
symbols = [(str(s).strip().upper() + ("" if str(s).strip().upper().endswith(".NS") else ".NS"))
           for s in df[sym_col] if str(s).strip()]

print(symbols)


['BSE.NS', 'BAJFINANCE.NS', 'BAJAJFINSV.NS', 'BDL.NS', 'BEL.NS', 'BHARTIARTL.NS', 'CHOLAFIN.NS', 'COFORGE.NS', 'DIVISLAB.NS', 'DIXON.NS', 'NYKAA.NS', 'HDFCBANK.NS', 'HDFCLIFE.NS', 'ICICIBANK.NS', 'INDHOTEL.NS', 'INDIGO.NS', 'KOTAKBANK.NS', 'MFSL.NS', 'MAXHEALTH.NS', 'MAZDOCK.NS', 'MUTHOOTFIN.NS', 'PAYTM.NS', 'PERSISTENT.NS', 'SBICARD.NS', 'SBILIFE.NS', 'SRF.NS', 'SHREECEM.NS', 'SOLARINDS.NS', 'TVSMOTOR.NS', 'UNITDSPR.NS']


/Users/hemank/Documents/github/.talib/lib/python3.12/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.niftyindices.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

"""
Scrape NSE: List of F&O Underlyings (Stocks) starting from:
https://www.nseindia.com/products-services/equity-derivatives-list-underlyings-information

Flow
----
1) Open the given page to get NSE cookies (anti-bot requires this warm-up).
2) From there, hop to "Contract Information" and auto-find the anchor
   "Permitted lot size (.csv)" (official file listing all F&O underlyings & lots).
3) Download the CSV, normalize columns, and filter to stock derivatives only:
   FUTSTK / OPTSTK (excludes index derivatives like NIFTY, BANKNIFTY, etc.)
4) Save outputs:
   - fno_stocks.csv  (symbol, lot_size, instrument, underlying)
   - fno_stocks.txt  (symbols only)

Why this route?
---------------
The "List of Underlyings and Information" page is client-side rendered and
does not expose a direct JSON. The official & maintained CSV link is surfaced
under "Contract Information" as "Permitted lot size (.csv)". Using that file
is the most stable way to scrape/collect the F&O underlyings programmatically.

Requirements
------------
pip install requests pandas beautifulsoup4

Author
------
GPT-5 — 2025-11-02 (IST)
"""

import io
import re
import time
import typing as t
import requests
import pandas as pd
from urllib.parse import urljoin
from bs4 import BeautifulSoup

START_URL = "https://www.nseindia.com/products-services/equity-derivatives-list-underlyings-information"
HOME_URL  = "https://www.nseindia.com/"

HDRS = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
}

# Fallback known archive path pattern if the on-page link isn’t directly found
ARCHIVE_HOSTS = [
    "https://nsearchives.nseindia.com/",
    "https://archives.nseindia.com/"  # historical alias sometimes used
]
CSV_CANDIDATES = [
    "content/fo/fo_mktlots.csv",        # Permitted lot size (current)
    "content/fo/fo_mktlots_*.csv",      # Dated variants if present
]

INDEX_TICKERS = {"NIFTY", "BANKNIFTY", "FINNIFTY", "MIDCPNIFTY", "NIFTYNXT50"}

def _get(session: requests.Session, url: str, timeout: int = 20) -> requests.Response:
    """GET with cookie warm-up and light retry/backoff."""
    backoff = 1.0
    for _ in range(5):
        try:
            r = session.get(url, headers=HDRS, timeout=timeout)
            if r.status_code == 200 and r.content:
                return r
        except requests.RequestException:
            pass
        # Warm cookies and backoff
        try:
            session.get(HOME_URL, headers=HDRS, timeout=timeout)
        except requests.RequestException:
            pass
        time.sleep(backoff)
        backoff = min(backoff * 2, 8.0)
    raise RuntimeError(f"Failed to GET {url}")

def _discover_contract_info_and_csv(session: requests.Session) -> str:
    """
    From START_URL, find the 'Contract Information' page and then locate
    the 'Permitted lot size (.csv)' link. Return CSV absolute URL.
    """
    # 1) Open start page (sets cookies)
    _get(session, START_URL)

    # 2) Try the Contract Information page directly (it’s the canonical hub)
    contract_info_url = "https://www.nseindia.com/products-services/equity-derivatives-contract-information"
    resp = _get(session, contract_info_url)
    soup = BeautifulSoup(resp.text, "html.parser")

    # 3) Find anchor whose text contains 'Permitted lot size' and ends with .csv
    a_tags = soup.find_all("a", href=True)
    for a in a_tags:
        text = (a.get_text() or "").strip().lower()
        href = a["href"]
        if "permitted" in text and "lot" in text and href.lower().endswith(".csv"):
            return urljoin(contract_info_url, href)

    # 4) Fallback: try known archive paths
    for host in ARCHIVE_HOSTS:
        for pattern in CSV_CANDIDATES:
            candidate = urljoin(host, pattern)
            try:
                r = _get(session, candidate)
                if r.status_code == 200 and r.content:
                    return candidate
            except Exception:
                continue

    raise RuntimeError("Could not locate 'Permitted lot size (.csv)' link.")

def _normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.rename(columns={c: c.strip().lower().replace(" ", "_") for c in df.columns})
    if "symbol" not in df.columns:
        # common alt: 'underlying'
        if "underlying" in df.columns:
            df["symbol"] = df["underlying"].astype(str).str.strip()
        else:
            # take first object column as symbol
            obj_cols = [c for c in df.columns if df[c].dtype == "object"]
            if obj_cols:
                df["symbol"] = df[obj_cols[0]].astype(str).str.strip()
    if "instrument" not in df.columns:
        for alt in ("inst", "inst_type", "instrument_type"):
            if alt in df.columns:
                df["instrument"] = df[alt]
                break
    if "lot_size" not in df.columns:
        for alt in ("market_lot", "mktlot", "fo_lot", "qty", "quantity_freeze"):
            if alt in df.columns:
                df["lot_size"] = df[alt]
                break
    if "underlying" not in df.columns and "symbol" in df.columns:
        df["underlying"] = df["symbol"]
    return df

def fetch_fno_stock_list() -> pd.DataFrame:
    with requests.Session() as s:
        # Warm cookies (helps with NSE bot-guard)
        try:
            s.get(HOME_URL, headers=HDRS, timeout=15)
        except requests.RequestException:
            pass

        csv_url = _discover_contract_info_and_csv(s)
        r = _get(s, csv_url)
        raw = r.content.decode("utf-8", errors="ignore")
        df = pd.read_csv(io.StringIO(raw))

    df = _normalize_columns(df)

    # Filter STOCK derivatives only: FUTSTK/OPTSTK. If instrument is missing,
    # exclude known index underlyings as a heuristic fallback.
    if "instrument" in df.columns:
        mask = df["instrument"].astype(str).str.contains("STK", case=False, na=False)
        out = df.loc[mask].copy()
    else:
        out = df.loc[~df["symbol"].astype(str).str.upper().isin(INDEX_TICKERS)].copy()

    out["symbol"] = out["symbol"].astype(str).str.upper().str.strip()
    if "lot_size" in out.columns:
        out["lot_size"] = pd.to_numeric(out["lot_size"], errors="coerce").astype("Int64")

    keep = [c for c in ["symbol", "lot_size", "instrument", "underlying"] if c in out.columns]
    out = out[keep].drop_duplicates().sort_values("symbol").reset_index(drop=True)
    return out

if __name__ == "__main__":
    try:
        print("Scraping NSE F&O underlyings (stocks) starting from the Underlyings page...")
        df = fetch_fno_stock_list()
        if df.empty:
            raise RuntimeError("Empty result after filtering FUTSTK/OPTSTK.")
        df.to_csv("fno_stocks.csv", index=False)
        df["symbol"].to_csv("fno_stocks.txt", index=False, header=False)
        print(f"✅ Done. Stocks in F&O: {len(df)}")
        print("📄 Saved: fno_stocks.csv, fno_stocks.txt")
    except Exception as e:
        print(f"❌ Error: {e}")


Scraping NSE F&O underlyings (stocks) starting from the Underlyings page...
✅ Done. Stocks in F&O: 213
📄 Saved: fno_stocks.csv, fno_stocks.txt
